In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ajc72_wo
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ajc72_wo


In [ ]:
# 필요한 모듈 로딩
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # 한국어 단발성 대화 데이터셋 불러오기 (학습용 데이터)
# # url : https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-009

# import pandas as pd
# chatbot_data = pd.read_excel('/content/drive/MyDrive/text_data/korean_short_talk.xlsx')

In [ ]:
# 2. 한국어 특정 문자 + 번역기 처리한 데이터 불러와서 학습
import pickle 

# 열기
path = '/content/drive/MyDrive/text_data/'

with open(path+'train_hanspell_new.pickle', 'rb') as f:
    chatbot_data = pickle.load(f)

In [ ]:
len(chatbot_data)

38594

In [ ]:
print(chatbot_data['Emotion'].value_counts())

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
중립    4830
Name: Emotion, dtype: int64


텍스트 교정 및 전처리한 데이터로 학습시키기

In [ ]:
# 모델 설정을 위한 변수 전처리 
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [ ]:
data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
# ['sentence', 'class'] 형태로 잘 넣어졌나 확인
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는 게 맞는 일인가요..??', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는 게 어려운데 해체를 한다고?', '1']
['당연히 그렇게 해야지 우리나라도 판매를 중단하라', '2']
['그 거들은 뒤부터 미치겠어요...', '3']
['최악의 상황 중 그나마 나은 방법이네. 기분은 잡치겠지만', '4']
['  요리하는 것이 숙제하는 것처럼 힘든 저에게 용기나 게 해주시고 할 수 있을 것 같은 희망을 주셔서 감사합니다!!', '5']
['와이프도 그렇고 댓글 다 볼 텐데 이휘재 좀 하차하라고 전해주세요', '6']


In [ ]:
# train & test set 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=1)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

28945
9649


In [ ]:
# KoBERT 입력 데이터로 만들기
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
# 이건 나중에 최적화 값 찾아봐야할 듯
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# 토큰화 실행
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
len(dataset_train[0][0])

32

In [ ]:
dataset_train[0]

['폭스콘과 함께 폭만하려는 샤프 심히 우려가 되네~~~~~!', '0']

In [ ]:
# 토큰화 패딩 처리 후 결과값 
data_train[0]

(array([   2, 4871, 6664, 7539, 5468, 4983, 4871, 6150, 7808,  517, 6546,
        7753, 3060, 7996, 3500, 1763, 5702,  517,  463,  517,  463,  517,
         463,  517,  463,  517,  463,  517,    5,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(30, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [ ]:
# torch 형식의 dataset 생성
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정해줘야함##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train_dataloader

In [ ]:
# # 최종 모델 학습시키기
# for e in range(num_epochs):
#     train_acc = 0.0
#     test_acc = 0.0
#     model.train()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
#         optimizer.zero_grad()
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         loss = loss_fn(out, label)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
#         optimizer.step()
#         scheduler.step()  # Update learning rate schedule
#         train_acc += calc_accuracy(out, label)
#         if batch_id % log_interval == 0:
#             print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
#     print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
#     model.eval()

#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         test_acc += calc_accuracy(out, label)

#     print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
# import pickle

# path = '/content/drive/MyDrive/text_data/'

# with open(path+'model_hanspell_new.pickle', 'wb') as f:
#     pickle.dump(model, f)

In [ ]:
# 학습한 model 열기
path = '/content/drive/MyDrive/text_data/'

with open(path+'model_hanspell_new.pickle', 'rb') as f:
    model = pickle.load(f)

In [ ]:
# 새로운 문장 테스트
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# # 예측 함수 생성
# # torch 데이터 타입으로 좀 더 쉽게 할 수 있을 거 같은데, 일단은 모르겠어서 하나씩 예측하는 함수로 실행
# def predict(predict_sentence):

#     data = [predict_sentence, '0']
#     dataset_another = [data]

#     another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
#     test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
#     model.eval()

#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)

#         valid_length= valid_length
#         label = label.long().to(device)

#         out = model(token_ids, valid_length, segment_ids)

#         output = out.detach().cpu().numpy()

#     return np.argmax(output)

In [ ]:
import pandas as pd

test = pd.DataFrame(dataset_test)
test.columns = ['sentence', 'label']
test

,sentence,label
0,너무 감사해요.,5
1,언제나 응원하고 있어요!,5
2,난 이제 그 사람 번호가 없거든요,3
3,자본주의에 재벌을 죽여야 한다고? ㅋㅋㅋㅋ,2
4,독도를 다케시마로 표기한 애플은 안 쓴다.,6
...,...,...
9644,이걸 기점으로 닭 그네 탄핵 후 구속 가자!!!,2
9645,알 바 끊길 가봐 여기들 또 몰려왔네 모두 수사해서 아르바이트 비 토해내게 해야 한다,6
9646,이상하다 못해 괴상한 대통령_ 그걸 싸고도는 새누리당도 큰 문제다.... 슬프다 슬퍼..,6
9647,조선시대 묵힌 정책을 하며 세금은 시대를 반영하며 천정부지로 올리며 서민은 구닥다리...,6


In [ ]:
# import pickle
# path = '/content/drive/MyDrive/text_data/'

# with open(path+'test_label_done.pickle', 'wb') as f:
#     pickle.dump(test, f)

In [ ]:
# path = '/content/drive/MyDrive/text_data/'

# with open(path+'test_label_done.pickle', 'rb') as f:
#     test = pickle.load(f)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
# 예측 함수 생성
# Setting parameters

max_len = 64
batch_size = 16


def predict_set(dataset_test):

    test_acc = 0.0

    tokenizer = get_tokenizer()
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

    data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

    test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

    out_list =[]

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        output = out.detach().cpu().tolist()

        test_acc += calc_accuracy(out, label)
        acc = test_acc / (batch_id+1)

        out_list.append(output)

    pd = sum(out_list,[])
    pd_list = pd_list = [np.argmax(i) for i in pd]
    return pd_list

In [ ]:
p_test = predict_set(dataset_test)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/604 [00:00<?, ?it/s]

In [ ]:
test['predict2'] = p_test

In [ ]:
test

,sentence,label,predict2
0,너무 감사해요.,5,5
1,언제나 응원하고 있어요!,5,5
2,난 이제 그 사람 번호가 없거든요,3,0
3,자본주의에 재벌을 죽여야 한다고? ㅋㅋㅋㅋ,2,1
4,독도를 다케시마로 표기한 애플은 안 쓴다.,6,6
...,...,...,...
9644,이걸 기점으로 닭 그네 탄핵 후 구속 가자!!!,2,2
9645,알 바 끊길 가봐 여기들 또 몰려왔네 모두 수사해서 아르바이트 비 토해내게 해야 한다,6,6
9646,이상하다 못해 괴상한 대통령_ 그걸 싸고도는 새누리당도 큰 문제다.... 슬프다 슬퍼..,6,3
9647,조선시대 묵힌 정책을 하며 세금은 시대를 반영하며 천정부지로 올리며 서민은 구닥다리...,6,6


모델 검증

In [ ]:
#!pip install sklearn

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import recall_score
# from pycaret.classification import *

In [ ]:
pred_label=test['predict2']
label_test_y=test['label'].astype('int64')

In [ ]:
pred_label.value_counts().sum()

9649

In [ ]:
label_test_y.value_counts().sum()

9649

In [ ]:
# accuracy
print(accuracy_score(label_test_y , pred_label))

0.5412996165405741


In [ ]:
# recall
print(recall_score(label_test_y , pred_label, average=None))

[0.59956236 0.53221477 0.4460682  0.62686567 0.41399663 0.77861035
 0.3633014 ]


In [ ]:
# f1score
f1_score(label_test_y, pred_label, labels=None, average='macro', sample_weight=None, zero_division='warn')

0.5365895666672609

In [ ]:
# # 댓글 데이터 적용해보기
path = '/content/drive/MyDrive/text_data/'

with open(path+'daum_comment_new.pickle', 'rb') as f:
    daum_c = pickle.load(f)
with open(path+'naver_comment_new.pickle', 'rb') as f:
    naver_c = pickle.load(f)

In [ ]:
e_dict = {0 : '공포', 1 : '놀람', 2 : '분노', 3 : '슬픔',
         4 : '중립', 5 : '행복', 6 : '혐오'}

In [ ]:
import pandas as pd

daum = pd.DataFrame({'sentence' : sum(daum_c,[])})
daum

,sentence
0,가해자가 촉법소년이라고 피해자가 되냐어째서 그런 해석의 나오냐죄를 지어놓고 촉법소년...
1,"촉법이라 가해자라고???? 뭔 댕댕이 소리래?청소년이 칼 들고 지르면, 칼 맞은 사..."
2,전동 킥보드 같은데.. 미성년자.. 촉법소년이면.. 당연히 무면허겠지.. 그럼 아예...
3,"12, 10세인 아이들이 전동 킥보드를 탄 게 문제의 원인 아닌가? 너무 어려서..."
4,촉법소년법부모가 돈 벌고 싶으면 어린 자식 시켜서 차에 들이 받으라고 하면 되는 법...
...,...
3817,대구도 조치 좀 하자
3818,이제 남은 무개념 조폭 같은 라이더 놈들말살 타이밍
3819,누구 머리에서 뇌에서 발생한 것인지 몰라도 사고 공화국 만들려고 환장했구나
3820,3시간 이내 공유 업체에 이동 자동차도 그렇게 해라


In [ ]:
# 모델 데이터 타입에 맞게 변환
def cm_transform(comment):
  data_list = []
  for q in comment['sentence'] :
    data = []
    data.append(q)
    data.append('0')

    data_list.append(data)
  return(data_list)

In [ ]:
daum_cm = cm_transform(daum)

In [ ]:
daum['predict'] = predict_set(daum_cm)
daum['emotion'] = daum['predict'].map(e_dict)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/239 [00:00<?, ?it/s]

In [ ]:
daum['emotion'].value_counts()

혐오    1112
분노     856
중립     750
놀람     444
공포     403
슬픔     177
행복      80
Name: emotion, dtype: int64

In [ ]:
naver = pd.DataFrame({'sentence' : sum(naver_c,[])})
naver

,sentence
0,다른 건 몰라도 공유 킥보드는 없애는 게 맞다 무면허에 나 헬멧에 짐승들 한 쌍이...
1,공유 킥보드는 안전도 안전이지만 생각보다 요금 셈. 가끔 신호 잘못 걸리면 그냥 ...
2,킥보드 충 극협
3,제발 어린아이 앞에 태우고 휘청휘청 인도로 다니지 좀 마라.
4,세금 낭비의 대표 사례. 박원숭이 얘는 진짜 최악의 행정 쓰레기.
...,...
46527,그냥 깔아버리고 싶습니다 독립군이 따로 있나요 밀정도 처단해야 합니다
46528,킥보드 타는 새끼들 보면 그지 상 그 시들이 99.9%이다
46529,뒤질 사람은 뒤져야지 별수 있나. 뒤질만한 짓을 하는데 안 뒤지는 게 이상하지.
46530,전동 킥라니 많이 좀 OOO라 그래야 법이 올바르게 바뀌지


In [ ]:
naver_cm = cm_transform(naver)
naver['predict'] = predict_set(naver_cm)
naver['emotion'] = naver['predict'].map(e_dict)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/2909 [00:00<?, ?it/s]

In [ ]:
naver['emotion'].value_counts()

혐오    15082
분노     9515
중립     8847
공포     5185
놀람     4156
슬픔     2546
행복     1201
Name: emotion, dtype: int64

In [ ]:
# 6개 부분 불러와서 적용

path = '/content/drive/MyDrive/text_data/section_pickle/'

with open(path+'economy_list.pickle', 'rb') as f:
    ec = pickle.load(f)

with open(path+'global_list.pickle', 'rb') as f:
    gb = pickle.load(f)
    
with open(path+'itscience_list.pickle', 'rb') as f:
    its = pickle.load(f)

with open(path+'lifeculture_list.pickle', 'rb') as f:
    lc = pickle.load(f)

with open(path+'politics_list.pickle', 'rb') as f:
    pol = pickle.load(f)

with open(path+'social_list.pickle', 'rb') as f:
    sc = pickle.load(f)

In [ ]:
def pic_cm(data):
  cm = pd.DataFrame({'sentence' : sum(data,[])})
  data_cm = cm_transform(cm)
  cm['predict'] = predict_set(data_cm)
  cm['emotion'] = cm['predict'].map(e_dict)
  return cm

In [ ]:
economy = pic_cm(ec)
global_ = pic_cm(gb)
it_science = pic_cm(its)
life_culture = pic_cm(lc)
politics = pic_cm(pol)
social = pic_cm(sc)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/408 [00:00<?, ?it/s]

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


  0%|          | 0/488 [00:00<?, ?it/s]

...

  0%|          | 0/880 [00:00<?, ?it/s]

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


  0%|          | 0/1386 [00:00<?, ?it/s]

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


  0%|          | 0/817 [00:00<?, ?it/s]

In [ ]:
economy['emotion'].value_counts()

혐오    2097
분노    1372
놀람    1030
중립     992
공포     478
슬픔     309
행복     249
Name: emotion, dtype: int64

In [ ]:
politics['emotion'].value_counts()

혐오    10264
분노     3470
놀람     3269
중립     2223
행복     1294
공포     1086
슬픔      562
Name: emotion, dtype: int64

In [ ]:
social['emotion'].value_counts()

혐오    4200
분노    3211
놀람    2033
공포    1442
중립     937
행복     688
슬픔     550
Name: emotion, dtype: int64

In [ ]:
it_science['emotion'].value_counts()

혐오    2706
놀람    2706
중립    1298
분노    1258
공포     997
행복     640
슬픔     222
Name: emotion, dtype: int64

In [ ]:
global_['emotion'].value_counts()

혐오    2396
놀람    1637
분노    1433
중립     920
공포     887
행복     323
슬픔     204
Name: emotion, dtype: int64

In [ ]:
life_culture['emotion'].value_counts()

혐오    4464
분노    2924
놀람    2227
공포    1923
중립    1414
행복     634
슬픔     480
Name: emotion, dtype: int64

예측값 전체 라벨 리스트로 확인해보기

In [ ]:
# 예측 함수 리스트 형식 생성
max_len = 64
batch_size = 16

def predict_set_list(dataset_test):

    test_acc = 0.0

    tokenizer = get_tokenizer()
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

    data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

    test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

    out_list =[]

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        output = out.detach().cpu().tolist()

        test_acc += calc_accuracy(out, label)
        acc = test_acc / (batch_id+1)

        out_list.append(output)

    return out_list

In [ ]:
def pic_cm_list(data):
  cm = pd.DataFrame({'sentence' : sum(data,[])})
  data_cm = cm_transform(cm)
  cm = predict_set_list(data_cm)
  return sum(cm,[])

In [ ]:
def final_label(data):
  comment_label = pic_cm_list(data)
  label_f = [np.argsort(i).tolist()[::-1][:len([x for x in i if x > 0])] for i in comment_label]

  data_f = pd.DataFrame({'sentence' : sum(data,[])})
  data_f['predict'] = label_f

  split_df = pd.DataFrame(data_f['predict'].tolist())

  data_final = pd.concat([data_f, split_df], axis = 1)

  return data_final

In [ ]:
economy = final_label(ec)
global_ = final_label(gb)
it_science = final_label(its)
life_culture = final_label(lc)
politics = final_label(pol)
social = final_label(sc)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/408 [00:00<?, ?it/s]

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


  0%|          | 0/488 [00:00<?, ?it/s]

...

  0%|          | 0/880 [00:00<?, ?it/s]

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


  0%|          | 0/1386 [00:00<?, ?it/s]

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


  0%|          | 0/817 [00:00<?, ?it/s]

In [ ]:
# filter
fil = (politics[0] == 5) & (politics[1] == 4)
fil
politics[fil].sample(n=30)

,sentence,predict,0,1,2,3,4
6245,"""저는 안후보님 지지합니다. 제 자신을 넘어 가족, 지인, 지역사회까지 뜨거운 지지...","[5, 4]",5,4.0,NaN,NaN,NaN
9967,"""안철수~파팅!!!""","[5, 4, 2]",5,4.0,2.0,NaN,NaN
21925,"""끝까지 응원해주세요""","[5, 4]",5,4.0,NaN,NaN,NaN
9118,"""윤석열이 공부는 잘하고 순수한 사람인듯... 허튼소리안하고 누구 뒷담도 안까고.....","[5, 4, 1]",5,4.0,1.0,NaN,NaN
13031,"""ㅋㅋㅋㅋ 완전 초중딩 반장선거 수준인듯 ㅋㅋㅋㅋ 하교전 매일 친구들 책상 닦겠다던...","[5, 4, 6]",5,4.0,6.0,NaN,NaN
18700,"""불타는 마음을 누르고 이른새벽 지하철역에서 인사를 하는건 그래도 정권교체를 하겠다...","[5, 4]",5,4.0,NaN,NaN,NaN
20681,"""토론에는 대본도 없고 코치 받아도 아는게 없는데 어찌 대답할래? 코미디프로보다 윤...","[5, 4]",5,4.0,NaN,NaN,NaN
16459,"""국민이 부른 윤석렬 국민에게만 충성한다""","[5, 4]",5,4.0,NaN,NaN,NaN
7874,"""이준석 힘내라 그리고 이나라 진짜우익의 진면모를 보여주길바란다""","[5, 4, 2, 6]",5,4.0,2.0,6.0,NaN
16941,"""윤석열 화이팅""","[5, 4]",5,4.0,NaN,NaN,NaN


In [ ]:
# 현재 데이터로 할수 있는거 ????

# -> 전처리한 댓글 데이터 -> pretrain된 kobert 모델에 넣어서 예측값 생성 -> 긍정 / 부정 여부 확인을 통해 부정의견의 비율이 얼마나 되는지 확인해보자